#HIDS-506 Final Project: Merging Datasets and One-hot-encoding

## Packages



We will start by loading some of the packages that will help us 
organize and visualize the data. Other packages will be loaded as necessary.

In [ ]:
#Import packages
from collections import Counter
import pandas as pd
import numpy as np
import collections, numpy
import mlxtend
import matplotlib 
from matplotlib import pyplot 
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
matplotlib.rcParams['figure.figsize'] = (10, 10)


from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
from sklearn.metrics import roc_curve, precision_recall_curve, auc

## Dataset

**Research Question:** Can machine learning (ML) provide a data-driven alternative to current emergency care triage and produce an accurate patient acuity score based on minimal patient information upon admission to the Emergency Department (ED), thus reducing human-biased medical decisions and preserving clinician time, attention, and ED workflow? 

Triage Dataset
* **Subject ID:** Unique patient identification number (will be dropped from feature set; used to merge datasets)
* **Temperature:** Triage vital sign; first temperature recording of patient upon ED arrival (Fahrenheit)
* **Heart Rate:** Triage vital sign; first record of patient heart rate upon arrival to ED (bpm)
* **Respiration Rate:** Triage vital sign; first record of patient respiration rate upon arrival to ED (breaths per minute)
* **Oxygen Saturation:** Triage vital sign; first record of patient oxygen saturation level - a measure of how much hemoglobin is currently bound to oxygen compared to how much hemoglobin remains unbound - upon arrival to ED (%)
* **Systolic Blood Pressure:** Triage vital sign; first record of patient systolic blood pressure upon arrival to ED (mmHg)
* **Diastolic Blood Pressure:** Triage vital sign; first record of patient diastolic blood pressure upon arrival to ED (mmHg)
* **Pain:** Triage vital sign; first record of patient reported pain upon arrival to ED (scale 0-10)
* **Chief Complaint:** Patient reported reason for admission to ED 
* **Outcome/Acuity:** Multi-class variable (1-5 patient condition severity rating upon arrival to the ED)

ED-Stays Dataset
* **Subject ID:** Unique patient identification number (will be dropped from feature set; used to merge datasets)
* **In-Time:** Arrival time of patient to ED (datetime)
* **Out-Time:** Discharge/transfer time out of ED (datetime)
* **Gender:** Male or Female
* **Race:** White, black/african american, hispanic/latinom asian, portugese, south american, american indian/alaska native, native hawaiian or other pacific islander 
* **Arrival Transport:** Walk-in, ambulance, unknown, other, helicopter 
* **Disposition:** Patient discharge/transfer location

MIMIC-IV 2.2 Admissions Dataset
* **Subject ID:** Unique patient identification number (will be dropped from feature set; used to merge datasets)
* **language:** Spoken language of patient
* **ED Registration Time:** Time of patient registration in the ED (datetime)
* **ED Out-Time:** Time of patient discharge/transfer
* **Insurance:** Medicare, Medicaid, or Other

MIMIC-IV 2.2 Patients Dataset
* **Subject ID:** Unique patient identification number (will be dropped from feature set; used to merge datasets)
* **Anchor Age:** Age of patient during anchor year (years) (de-identified encoding)
* **Anchor Year:** Year range patient was seen in the ED (years) (de-identified encoding)



Note: After EDA and merging data sets, the admissions data set was no longer used. 

Exploring the datasets:

In [ ]:
#from google.colab import 
#drivedrive.mount('/content/drive')

In [ ]:
#mounting google drive to access files 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Merging Datasets

In [ ]:
# admissions file includes data from MIMIC-IV, patients will only be included if seen in the ED
# defining admissions file path 
admissions_file = '/content/drive/MyDrive/HIDS_506_FinalProject/data/admissions_clean.csv'

# read in admissions file to dataframe
admissions_df = pd.read_csv(admissions_file)


# patients file includes data from MIMIC-IV, patients will only be included if seen in the ED
# defining patients file path 
patients_file = '/content/drive/MyDrive/HIDS_506_FinalProject/data/patients_clean.csv'

# read in patients file to dataframe
patients_df = pd.read_csv(patients_file)


#reading in triage csv file
triage_file = "/content/drive/MyDrive/HIDS_506_FinalProject/data/triage_clean.csv"

#assigning triage csv file to dataframe
triage_df = pd.read_csv(triage_file)

# defining edstays file path
edstays_file = "/content/drive/MyDrive/HIDS_506_FinalProject/data/edstays_clean.csv"

# reading edstays file into dataframe
edstays_df = pd.read_csv(edstays_file)

In [ ]:
admissions_df.columns

admissions_df = admissions_df[['subject_id', 'insurance', 'edregtime', 'edouttime']]
admissions_df.head()

subject_id insurance            edregtime            edouttime
0    10000032     Other  2180-05-06 19:17:00  2180-05-06 23:30:00
1    10000032  Medicaid  2180-06-26 15:54:00  2180-06-26 21:31:00
2    10000032  Medicaid  2180-08-05 20:58:00  2180-08-06 01:44:00
3    10000032  Medicaid  2180-07-23 05:54:00  2180-07-23 14:00:00
4    10000068     Other  2160-03-03 21:55:00  2160-03-04 06:26:00

In [ ]:
# merging datasets into one matching on subject id
triage_clean_df = triage_df.merge(edstays_df, how='inner', on='subject_id')

In [ ]:
triage_clean_df = triage_clean_df.merge(patients_df, how='inner', on='subject_id')

In [ ]:
#triage_clean_df = triage_clean_df.merge(admissions_df, how='inner', on='subject_id')

In [ ]:
triage_clean_df.head()

Unnamed: 0_x  subject_id  stay_id_x  temperature  heartrate  resprate  \
0             0    10000032   32952584         97.8       87.0      14.0   
1             0    10000032   32952584         97.8       87.0      14.0   
2             0    10000032   32952584         97.8       87.0      14.0   
3             0    10000032   32952584         97.8       87.0      14.0   
4             0    10000032   32952584         97.8       87.0      14.0   

   o2sat   sbp   dbp  pain  ...  stay_id_y               intime  \
0   97.0  71.0  43.0     7  ...   33258284  2180-05-06 19:17:00   
1   97.0  71.0  43.0     7  ...   38112554  2180-06-26 15:54:00   
2   97.0  71.0  43.0     7  ...   35968195  2180-08-05 20:58:00   
3   97.0  71.0  43.0     7  ...   32952584  2180-07-22 16:24:00   
4   97.0  71.0  43.0     7  ...   39399961  2180-07-23 05:54:00   

               outtime  gender   race arrival_transport disposition  \
0  2180-05-06 23:30:00       F  WHITE         AMBULANCE    ADMITTED   
1  2180-06-26 21:31:00       F  WHITE         AMBULANCE    ADMITTED   
2  2180-08-06 01:44:00       F  WHITE         AMBULANCE    ADMITTED   
3  2180-07-23 05:54:00       F  WHITE         AMBULANCE        HOME   
4  2180-07-23 14:00:00       F  WHITE         AMBULANCE    ADMITTED   

  Unnamed: 0  age_category deceased  
0          0  middle_adult        1  
1          0  middle_adult        1  
2          0  middle_adult        1  
3          0  middle_adult        1  
4          0  middle_adult        1  

[5 rows x 24 columns]

In [ ]:
triage_clean_df.columns

Index(['Unnamed: 0_x', 'subject_id', 'stay_id_x', 'temperature', 'heartrate',
       'resprate', 'o2sat', 'sbp', 'dbp', 'pain', 'acuity', 'chiefcomplaint',
       'Unnamed: 0_y', 'hadm_id', 'stay_id_y', 'intime', 'outtime', 'gender',
       'race', 'arrival_transport', 'disposition', 'Unnamed: 0',
       'age_category', 'deceased'],
      dtype='object')

In [ ]:
triage_clean_df = triage_clean_df[['subject_id', 'temperature', 'heartrate', 'resprate', 'o2sat', 'sbp', 'dbp', 'pain', 'acuity', 'chiefcomplaint', 'intime', 'outtime', 'gender', 'race', 'arrival_transport', 'disposition', 'age_category', 'deceased']]

In [ ]:
# 18 features (including acuity outcome feature), 2984649 observations
triage_clean_df.shape

(2984649, 18)

In [ ]:
triage_clean_df.head()

subject_id  temperature  heartrate  resprate  o2sat   sbp   dbp  pain  \
0    10000032         97.8       87.0      14.0   97.0  71.0  43.0     7   
1    10000032         97.8       87.0      14.0   97.0  71.0  43.0     7   
2    10000032         97.8       87.0      14.0   97.0  71.0  43.0     7   
3    10000032         97.8       87.0      14.0   97.0  71.0  43.0     7   
4    10000032         97.8       87.0      14.0   97.0  71.0  43.0     7   

   acuity chiefcomplaint               intime              outtime gender  \
0     2.0    Hypotension  2180-05-06 19:17:00  2180-05-06 23:30:00      F   
1     2.0    Hypotension  2180-06-26 15:54:00  2180-06-26 21:31:00      F   
2     2.0    Hypotension  2180-08-05 20:58:00  2180-08-06 01:44:00      F   
3     2.0    Hypotension  2180-07-22 16:24:00  2180-07-23 05:54:00      F   
4     2.0    Hypotension  2180-07-23 05:54:00  2180-07-23 14:00:00      F   

    race arrival_transport disposition  age_category  deceased  
0  WHITE         AMBULANCE    ADMITTED  middle_adult         1  
1  WHITE         AMBULANCE    ADMITTED  middle_adult         1  
2  WHITE         AMBULANCE    ADMITTED  middle_adult         1  
3  WHITE         AMBULANCE        HOME  middle_adult         1  
4  WHITE         AMBULANCE    ADMITTED  middle_adult         1

In [ ]:
# according to mimic-iv-ed 1 indicates the highest severity and 5 indicates the lowest severity.
# in real life, 1 indicates a stable patient and 5 indicates highest severity and most urgent 

triage_clean_df['acuity'].value_counts()

3.0    1776350
2.0     930618
4.0     181285
1.0      87155
5.0       9241
Name: acuity, dtype: int64

In [ ]:
triage_clean_df.isnull().sum()

subject_id            0
temperature           0
heartrate             0
resprate              0
o2sat                 0
sbp                   0
dbp                   0
pain                  0
acuity                0
chiefcomplaint       68
intime                0
outtime               0
gender                0
race                  0
arrival_transport     0
disposition           0
age_category          0
deceased              0
dtype: int64

In [ ]:
# converting intime and outtime to data typr datetime varaibles 

triage_clean_df['intime'] = pd.to_datetime(triage_clean_df['intime'])
triage_clean_df['outtime'] = pd.to_datetime(triage_clean_df['outtime'])

In [ ]:
triage_clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2984649 entries, 0 to 2984648
Data columns (total 18 columns):
 #   Column             Dtype         
---  ------             -----         
 0   subject_id         int64         
 1   temperature        float64       
 2   heartrate          float64       
 3   resprate           float64       
 4   o2sat              float64       
 5   sbp                float64       
 6   dbp                float64       
 7   pain               int64         
 8   acuity             float64       
 9   chiefcomplaint     object        
 10  intime             datetime64[ns]
 11  outtime            datetime64[ns]
 12  gender             object        
 13  race               object        
 14  arrival_transport  object        
 15  disposition        object        
 16  age_category       object        
 17  deceased           int64         
dtypes: datetime64[ns](2), float64(7), int64(3), object(6)
memory usage: 432.7+ MB


In [ ]:
Counter(triage_clean_df['chiefcomplaint'])

Counter({'Hypotension': 3433,
         'Abd pain, Abdominal distention': 2064,
         'n/v/d, Abd pain': 1948,
         'Abdominal distention': 2596,
         'Confusion, Hallucinations': 12,
         'Altered mental status, B Pedal edema': 2,
         'L CHEEK ABSCESS': 3,
         'LEFT CHEEK SWELLING, Abscess': 3,
         'L FACIAL SWELLING': 94,
         'Suture removal': 1026,
         'Laceration, s/p Fall': 876,
         'Head injury': 2545,
         'Throat foreign body sensation': 992,
         'L Hip pain': 5471,
         'R Foot pain': 11863,
         'ANEMIA S/P FALL': 1,
         'Abd pain': 94176,
         'LUQ abd pain': 2271,
         'ILI': 13117,
         'Vomiting': 4533,
         'ETOH': 129812,
         'Dizziness': 15332,
         'L Shoulder pain, Visual changes': 2,
         's/p Fall': 26970,
         'Abnormal labs, Calf pain': 1,
         'Chest pain': 98568,
         'WEAKNESS': 5739,
         'VOMITING AND/OR NAUSEA': 3290,
         'L LEG SWELLING': 228

In [ ]:
triage_clean_df['chiefcomplaint'].value_counts()

ETOH                        129812
Chest pain                   98568
Abd pain                     94176
ETOH, Unable to ambulate     67363
SI                           53002
                             ...  
Fever, R Finger injury           1
DYSPNEA X 10 DAYS                1
psychosis                        1
Ankle injury, MVC                1
? SEIZURE TODAY                  1
Name: chiefcomplaint, Length: 55431, dtype: int64

In [ ]:
# re-coding categorical variables 
triage_clean_df['gender'].replace({'M': 0, 'F':1}, inplace=True)

In [ ]:
triage_clean_df.tail()

subject_id  temperature  heartrate  resprate  o2sat    sbp   dbp  \
2984644    19999828         98.1       83.0      18.0  100.0  107.0  75.0   
2984645    19999828         98.1       83.0      18.0  100.0  107.0  75.0   
2984646    19999828         96.6      112.0      18.0  100.0  110.0  82.0   
2984647    19999828         96.6      112.0      18.0  100.0  110.0  82.0   
2984648    19999914         99.5       81.0      10.0  100.0   93.0  55.0   

         pain  acuity                        chiefcomplaint  \
2984644     8     2.0                  Abd pain, Wound eval   
2984645     8     2.0                  Abd pain, Wound eval   
2984646     4     2.0                  Abd pain, Wound eval   
2984647     4     2.0                  Abd pain, Wound eval   
2984648     0     2.0  Altered mental status, Substance use   

                     intime             outtime  gender           race  \
2984644 2149-01-08 09:11:00 2149-01-08 18:12:00       1          WHITE   
2984645 2147-07-17 17:18:00 2147-07-18 17:34:00       1          WHITE   
2984646 2149-01-08 09:11:00 2149-01-08 18:12:00       1          WHITE   
2984647 2147-07-17 17:18:00 2147-07-18 17:34:00       1          WHITE   
2984648 2158-12-24 11:41:00 2158-12-24 11:56:00       1  OTHER/UNKNOWN   

        arrival_transport disposition  age_category  deceased  
2984644         AMBULANCE    ADMITTED  middle_adult         0  
2984645           WALK IN    ADMITTED  middle_adult         0  
2984646         AMBULANCE    ADMITTED  middle_adult         0  
2984647           WALK IN    ADMITTED  middle_adult         0  
2984648           UNKNOWN      ELOPED  middle_adult         0

In [ ]:
# changing all cheif complaints to be lower case so that unique values can be found
triage_clean_df['chiefcomplaint'] = triage_clean_df['chiefcomplaint'].astype(str)
triage_clean_df['chiefcomplaint'] = triage_clean_df['chiefcomplaint'].apply(str.lower)


# import and instantiate CountVectorizer (with the default parameters)
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

# convert the column to a numpy array and concatenate the strings
arr = np.concatenate(triage_clean_df[['chiefcomplaint']].to_numpy())

# print the resulting array
print(arr)


['hypotension' 'hypotension' 'hypotension' ... 'abd pain, wound eval'
 'abd pain, wound eval' 'altered mental status, substance use']


In [ ]:
triage_clean_df['chiefcomplaint'].value_counts()

abd pain                            132285
etoh                                129812
chest pain                          120942
etoh, unable to ambulate             67399
dyspnea                              63779
                                     ...  
s/p mcc                                  1
transfer, elev trop                      1
low speed mvc                            1
headache, dizziness, head injury         1
? seizure today                          1
Name: chiefcomplaint, Length: 54053, dtype: int64

In [ ]:
# most common cheif complaints 
counter = Counter(arr)

most_common = counter.most_common()
print(most_common)

[('abd pain', 132285), ('etoh', 129812), ('chest pain', 120942), ('etoh, unable to ambulate', 67399), ('dyspnea', 63779), ('si', 53002), ('s/p fall', 39588), ('headache', 35688), ('back pain', 33896), ('wound eval', 32545), ('unable to ambulate', 27462), ('abd pain, n/v', 25142), ('weakness', 23869), ('fever', 21900), ('n/v', 20933), ('dizziness', 20322), ('lower back pain', 18727), ('chest pain, dyspnea', 18426), ('cough', 18189), ('brbpr', 15979), ('abdominal pain', 15385), ('abnormal labs', 15102), ('seizure', 13398), ('n/v/d', 13183), ('ili', 13117), ('altered mental status', 13057), ('r foot pain', 12684), ('hyperglycemia', 11518), ('palpitations', 11507), ('syncope', 11354), ('cp', 11071), ('rash', 10752), ('r flank pain', 10501), ('sore throat', 10316), ('dysuria', 9530), ('neck pain', 9114), ('rlq abdominal pain', 9098), ('sob', 8964), ('l foot pain', 8816), ('abd pain, n/v/d', 8770), ('diarrhea', 8708), ('abd pain, vomiting', 8469), ('l knee pain', 8412), ('r knee pain', 8372)

In [ ]:
# 54053 possible chief complaints 
len(counter)

54053

In [ ]:
triage_clean_df.tail()

subject_id  temperature  heartrate  resprate  o2sat    sbp   dbp  \
2984644    19999828         98.1       83.0      18.0  100.0  107.0  75.0   
2984645    19999828         98.1       83.0      18.0  100.0  107.0  75.0   
2984646    19999828         96.6      112.0      18.0  100.0  110.0  82.0   
2984647    19999828         96.6      112.0      18.0  100.0  110.0  82.0   
2984648    19999914         99.5       81.0      10.0  100.0   93.0  55.0   

         pain  acuity                        chiefcomplaint  \
2984644     8     2.0                  abd pain, wound eval   
2984645     8     2.0                  abd pain, wound eval   
2984646     4     2.0                  abd pain, wound eval   
2984647     4     2.0                  abd pain, wound eval   
2984648     0     2.0  altered mental status, substance use   

                     intime             outtime  gender           race  \
2984644 2149-01-08 09:11:00 2149-01-08 18:12:00       1          WHITE   
2984645 2147-07-17 17:18:00 2147-07-18 17:34:00       1          WHITE   
2984646 2149-01-08 09:11:00 2149-01-08 18:12:00       1          WHITE   
2984647 2147-07-17 17:18:00 2147-07-18 17:34:00       1          WHITE   
2984648 2158-12-24 11:41:00 2158-12-24 11:56:00       1  OTHER/UNKNOWN   

        arrival_transport disposition  age_category  deceased  
2984644         AMBULANCE    ADMITTED  middle_adult         0  
2984645           WALK IN    ADMITTED  middle_adult         0  
2984646         AMBULANCE    ADMITTED  middle_adult         0  
2984647           WALK IN    ADMITTED  middle_adult         0  
2984648           UNKNOWN      ELOPED  middle_adult         0

In [ ]:
# Create dummy columns for categorical variables
prefix_cols = ['race', 'arrival_transport', 'disposition', 'age_category']
dummy_cols = ['race', 'arrival_transport', 'disposition', 'age_category']
# one-hot-encode categorical variables using dummies 
triage_clean_df = pd.get_dummies(triage_clean_df, prefix=prefix_cols, columns=dummy_cols)

In [ ]:
triage_clean_df.head()


subject_id  temperature  heartrate  resprate  o2sat   sbp   dbp  pain  \
0    10000032         97.8       87.0      14.0   97.0  71.0  43.0     7   
1    10000032         97.8       87.0      14.0   97.0  71.0  43.0     7   
2    10000032         97.8       87.0      14.0   97.0  71.0  43.0     7   
3    10000032         97.8       87.0      14.0   97.0  71.0  43.0     7   
4    10000032         97.8       87.0      14.0   97.0  71.0  43.0     7   

   acuity chiefcomplaint  ... disposition_ELOPED disposition_EXPIRED  \
0     2.0    hypotension  ...                  0                   0   
1     2.0    hypotension  ...                  0                   0   
2     2.0    hypotension  ...                  0                   0   
3     2.0    hypotension  ...                  0                   0   
4     2.0    hypotension  ...                  0                   0   

   disposition_HOME  disposition_LEFT AGAINST MEDICAL ADVICE  \
0                 0                                        0   
1                 0                                        0   
2                 0                                        0   
3                 1                                        0   
4                 0                                        0   

   disposition_LEFT WITHOUT BEING SEEN  disposition_OTHER  \
0                                    0                  0   
1                                    0                  0   
2                                    0                  0   
3                                    0                  0   
4                                    0                  0   

   disposition_TRANSFER  age_category_middle_adult  age_category_senior  \
0                     0                          1                    0   
1                     0                          1                    0   
2                     0                          1                    0   
3                     0                          1                    0   
4                     0                          1                    0   

   age_category_young_adult  
0                         0  
1                         0  
2                         0  
3                         0  
4                         0  

[5 rows x 39 columns]

Note: chief complaints will be one-hot-encoded in the next notebook due to storage constraints 

In [ ]:
triage_clean_df.to_csv('/content/drive/MyDrive/HIDS_506_FinalProject/data/triage_clean.csv')